In [1]:
import numpy as np
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import chi2_contingency
from scipy import stats

In [2]:
# x=pd.read_csv('../_xlable4_withNa_AllColumns.txt')
# y=pd.read_csv('../_targelable4_withNa_AllColumns.txt')

# y=y[['Unnamed: 0','L100800', ]]

In [3]:
# data= pd.merge(x,y, how='inner',left_on='Unnamed: 0', right_on='Unnamed: 0')
data= pd.read_csv('../sep19SexAndAgeAddedFINAL DATASET_ver2.txt',sep=',')
data['max']=np.min(data[['FIELD_6','FIELD_7']],axis=1)
data=data.drop(columns=['FIELD_6','FIELD_7'])

In [ ]:
# data=data.query('FIELD_15 !=1 and FIELD_17 !=1 and FIELD_22 != 1 and FIELD_24 != 1 and FIELD_16 != 1 and FIELD_23 != 1')
# data=data.query('FIELD_16 != 1 and FIELD_23 != 1')

In [4]:
# data=data[data.SEX==0]

data=data[data.FIELD_16!=1]
data=data[data.FIELD_23!=1]
data=data[data.FIELD_15!=1]
data=data[data.FIELD_17!=1]
data=data[data.FIELD_22!=1]
data=data[data.FIELD_24!=1]

# data=data[data.AGE>=40]
# data=data[data.AGE<50]

print(data.shape)

(466680, 405)


In [6]:
df=data.copy()
conditions3 = [
    (df.L100800 < 100)  ,
    (df.L100800 >= 100) & (df.L100800 < 126),
    (df.L100800 >= 126)]
choices3 = [0,1,2]
df['CLASS'] = np.select(conditions3, choices3, default=0)
data=df.copy()
data.head()

Unnamed: 0  FIELD_1   FIELD_2          COMPARE   AGE     B_DAY  SEX  \
0           0  1000003  20130913  100000320130913  46.0  670212.0  0.0   
1           1  1000003  20140613  100000320140613  47.0  670212.0  0.0   
2           2  1000003  20150917  100000320150917  48.0  670212.0  0.0   
3           3  1000003  20160617  100000320160617  49.0  670212.0  0.0   
4           4  1000003  20170608  100000320170608  50.0  670212.0  0.0   

  FIELD_3 FIELD_4 FIELD_5  ... S005200  S007400 S008501 S008502 S008503  \
0     NaN       4       1  ...     NaN      NaN     NaN     NaN     NaN   
1     NaN       4       1  ...     NaN      NaN     NaN     NaN     NaN   
2       1       4       1  ...     NaN      NaN     NaN     NaN     NaN   
3     NaN       4     NaN  ...     NaN      NaN     NaN     NaN     NaN   
4     NaN       4     NaN  ...     NaN      NaN     NaN     NaN     NaN   

   S008504  S010101  S010102  max  CLASS  
0      NaN     18.0     14.0  NaN      1  
1      NaN     15.0     16.0  NaN      1  
2      NaN     16.0     16.0  NaN      1  
3      NaN     16.0     18.0  NaN      1  
4      NaN     16.0     18.0  NaN      1  

[5 rows x 406 columns]

In [7]:
#split column types to categorical and numerical
numerical_cols = list(data.columns[~data.columns.str.startswith('FIELD')])
categorical_cols = list(data.columns[data.columns.str.startswith('FIELD')])
categorical_cols.append('CLASS')

Calculate correlation Value - for numerical values
========

In [8]:
corr=data[numerical_cols].corr()
corr=corr.L100800_y
corrvalPD=pd.DataFrame()
corrvalPD['Col']=corr.index
corrvalPD['val']=abs(corr.values)

corrvalPD=corrvalPD.sort_values(by='val',ascending=False)
corrvalPD.head()

Col       val
277    CLASS  1.000000
149  L501600  1.000000
153  L502000  0.956054
118  L303204  0.774597
115  L303201  0.774597

In [ ]:
#count of none NA values of feature set
mydataset=data.copy()[numerical_cols]
colCount=[]
for i in mydataset.columns:
    colCount.append([i,mydataset[i].dropna().shape[0]])

colCountPD=pd.DataFrame(colCount,columns=['Col','Count'])
colCountPD.head()

In [ ]:
mergedColCorrCount=pd.merge(corrvalPD,colCountPD,how='inner',left_on='Col',right_on='Col')
mergedColCorrCount=mergedColCorrCount[mergedColCorrCount.Count>50000]
mergedColCorrCount=mergedColCorrCount.sort_values(by='val', ascending=False)
mergedColCorrCount.head(20)

In [ ]:
print(list(mergedColCorrCount.head(15).Col))

Anova test - For categorical values [Questionnaire answers]
=======

In [ ]:
mydataset=data.copy()[categorical_cols] #[np.insert(categorical_cols,0,'L100800')]

# mydataset=mydataset.drop(columns=['FIELD_1','FIELD_2','FIELD_8','FIELD_10','FIELD_11','FIELD_12','FIELD_39','FIELD_88',
#                            'FIELD_89','FIELD_109','FIELD_110','FIELD_111','FIELD_70','FIELD_82','FIELD_85','FIELD_91',
#                            'FIELD_103','FIELD_106'])

# mydataset=mydataset.drop(columns=['FIELD_1','FIELD_2','FIELD_87','FIELD_8','FIELD_10','FIELD_11','FIELD_12','FIELD_39',
#                                   'FIELD_88','FIELD_89', 'FIELD_108','FIELD_109','FIELD_110','FIELD_111','FIELD_82',
#                                  'FIELD_91','FIELD_103','FIELD_118','FIELD_119','FIELD_120','FIELD_121','FIELD_122',
#                                  'FIELD_123','FIELD_124','FIELD_125','FIELD_126','FIELD_127','FIELD_128','FIELD_129',
#                                  'FIELD_130','FIELD_131','FIELD_132','FIELD_133','FIELD_134','FIELD_135','FIELD_136',
#                                   'FIELD_137','FIELD_138','FIELD_139','FIELD_140'])


mydataset=mydataset.drop(columns=['FIELD_1','FIELD_2','FIELD_87','FIELD_8','FIELD_10','FIELD_11','FIELD_12','FIELD_39',
                                  'FIELD_88','FIELD_89', 'FIELD_108','FIELD_109','FIELD_110','FIELD_111','FIELD_82',
                                 'FIELD_91','FIELD_103','FIELD_118','FIELD_119','FIELD_120','FIELD_121','FIELD_122',
                                 'FIELD_123','FIELD_124','FIELD_125','FIELD_126','FIELD_127','FIELD_128','FIELD_129',
                                 'FIELD_130','FIELD_131','FIELD_132','FIELD_133','FIELD_134','FIELD_135','FIELD_136',
                                  'FIELD_137','FIELD_138','FIELD_139','FIELD_140',
                                  'FIELD_64','FIELD_65','FIELD_66','FIELD_67','FIELD_68','FIELD_69','FIELD_70','FIELD_72',
                                  'FIELD_73','FIELD_74','FIELD_75','FIELD_76','FIELD_77','FIELD_80','FIELD_81','FIELD_84',
                                  'FIELD_85','FIELD_90','FIELD_93','FIELD_94','FIELD_95','FIELD_96','FIELD_97','FIELD_98',
                                  'FIELD_101','FIELD_102','FIELD_105','FIELD_106','FIELD_112','FIELD_113','FIELD_114',
                                  'FIELD_115','FIELD_116','FIELD_117','FIELD_118','FIELD_119','FIELD_9'
                                 ])


mydataset=mydataset.drop(mydataset[mydataset.FIELD_3=='`'].index)
mydataset=mydataset.drop(mydataset[mydataset.FIELD_3=='G'].index)


mydataset=mydataset.drop(mydataset[mydataset.FIELD_4=='.'].index)
mydataset=mydataset.drop(mydataset[mydataset.FIELD_5=='.'].index)

mydataset=mydataset.drop(mydataset[mydataset.FIELD_35=='?'].index)
mydataset=mydataset.drop(mydataset[mydataset.FIELD_35=='.'].index)
mydataset=mydataset.drop(mydataset[mydataset.FIELD_36=='.'].index)
mydataset=mydataset.drop(mydataset[mydataset.FIELD_37=='\\'].index)
 
# mydataset=mydataset.drop(mydataset[mydataset.FIELD_39 == '7+' ].index)
# mydataset=mydataset.drop(mydataset[mydataset.FIELD_39 == '5~7'].index)
# mydataset=mydataset.drop(mydataset[mydataset.FIELD_39 == '3-4'].index)


# mydataset=mydataset.drop(mydataset[mydataset.FIELD_95=='.'].index)

In [ ]:
colslist=mydataset.columns
DiabeticColVals=data.L100800_y

cor=[]
pval=[]
count=[]

for i in range(len(colslist)):
    print(i,colslist[i])
    
    temp=pd.DataFrame([])
    temp['a']=DiabeticColVals
    temp['b']=mydataset[colslist[i]].astype(float)
    temp=temp.dropna()
    count.append(temp.shape[0])
    
    uniquevalues=temp.b.unique()
    
    selectedGroupVals=[]
    for k in range(len(uniquevalues)):
        selectedGroupVals.append(np.asarray(temp[temp['b']==uniquevalues[k]].a))

    F, p = stats.f_oneway(*selectedGroupVals)
    cor.append(F)
    pval.append(p)
    

In [ ]:
FvalPvalCorr=pd.DataFrame()    
FvalPvalCorr['Cols']=colslist
FvalPvalCorr['F']=cor
FvalPvalCorr['P']=pval
FvalPvalCorr['Count']=count
FvalPvalCorr=FvalPvalCorr.sort_values(by='F', ascending=False)


In [ ]:
FvalPvalCorr[FvalPvalCorr.Count>50000].head(10)

In [ ]:
print(list(FvalPvalCorr[FvalPvalCorr.Count>50000].Cols)[:10])

========================= LASSO method
================

In [ ]:
selectedcols=['L100800_y', 'L100800_x', 'L104600', 'S000300', 'L103000', 'L103300', 'SEX', 'S000501', 'L101700', 
              'L190400', 'S000502', 'L190500', 'L101300', 'L190300', 'AGE','FIELD_33', 'FIELD_38', 'FIELD_31', 
              'FIELD_40', 'FIELD_27', 'FIELD_29', 'FIELD_41', 'FIELD_21', 'FIELD_5', 'FIELD_14'
             ]
data2=data[selectedcols].copy().dropna()
print(data2.shape)

In [ ]:
x=data2[['L100800_x', 'L104600', 'S000300', 'L103000', 'L103300', 'SEX', 'S000501', 'L101700', 
              'L190400', 'S000502', 'L190500', 'L101300', 'L190300', 'AGE','FIELD_33', 'FIELD_38', 'FIELD_31', 
              'FIELD_40', 'FIELD_27', 'FIELD_29', 'FIELD_41', 'FIELD_21', 'FIELD_5', 'FIELD_14']]
y=data2.L100800_y

In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(x,y)

print(clf.coef_)

print(clf.intercept_)  

# pd.DataFrame([x.columns.ravel(),clf.coef_.ravel()],columns=['Name','coeff'])

lassod=pd.DataFrame()
lassod['Name']=x.columns
lassod['coeff']=clf.coef_
lassod

lassod=lassod.drop(lassod[lassod.coeff==0].index)
lassod.coeff=abs(lassod.coeff)
lassod=lassod.sort_values(by='coeff', ascending=False)
lassod

In [ ]:
print(list(lassod.Name[:16]))

=============================== SelectKBest method
========================

In [ ]:
# from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectKBest

bestfeatures = SelectKBest(score_func=f_regression, k=10)
fit = bestfeatures.fit(x,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns

# x=x[x.columns[:15]]
print(list(x.columns[:16]))

========================================= LinearSVC
===============

In [ ]:
# from sklearn.svm import LinearSVC
# from sklearn.datasets import load_iris
# from sklearn.feature_selection import SelectFromModel

# lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(x, y)
# model = SelectFromModel(lsvc, prefit=True)
# X_new = model.transform(x)
# X_new.shape

In [ ]:
# X_new

================================ SequentialFeatureSelector 
================================

In [ ]:
# from sklearn.linear_model import LinearRegression
# from mlxtend.feature_selection import SequentialFeatureSelector as SFS
# from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
# from sklearn.ensemble import RandomForestRegressor

# lr = RandomForestRegressor()

# sfs = SFS(lr, 
#           k_features=13, 
#           forward=True, 
#           floating=False, 
#           scoring='neg_mean_squared_error',
#           cv=5)

# sfs = sfs.fit(x, y)
# fig = plot_sfs(sfs.get_metric_dict(), kind='std_err')

# plt.title('Sequential Forward Selection (w. StdErr)')
# plt.grid()
# plt.show()

================================= feature_importances
=====

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(x,y)

print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers

#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=x.columns)
feat_importances.nlargest(16).plot(kind='barh')
plt.show()

print(feat_importances.nlargest(16))

============================= Backward Elimination
========

In [ ]:
import statsmodels.regression.linear_model as sm
temp=x.copy()
temp['const']=np.ones((x.shape[0],1))
regressor_OLS = sm.OLS(endog = y, exog = temp).fit()
regressor_OLS.summary()

In [ ]:
temp=temp.drop(columns=['L103000','L103300','L190300','FIELD_40','FIELD_27','FIELD_29','FIELD_21','FIELD_5','FIELD_14'])
regressor_OLS = sm.OLS(endog = y, exog = temp).fit()
regressor_OLS.summary()

In [ ]:
temp=temp.drop(columns=['FIELD_33'])
regressor_OLS = sm.OLS(endog = y, exog = temp).fit()
regressor_OLS.summary()

In [ ]:
temp.columns